# CMSC320 Fall 2022 Final Project- Riya Kukadia and Amartya Patel

In [27]:
# Topic- Analysing UFO Sightings around the world.

# Contributors: Riya Kukadia () and Amartya Patel (117183986)

In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
#import geopandas

import sys
import i

In [26]:
# Step 1: Data Collection/Curation
# Since we are using a ready to use dataset, there is no need to parse the data.
# We have obtained this dataset from kaggle.com
# Link:
UFO_Dataframe = pd.read_csv('scrubbed.csv', low_memory=False)

UFO_Dataframe.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [ ]:
# Step 2: Data management/representation

# Dealing with missing locations
UFO_Dataframe = UFO_Dataframe.fillna('unknown')

#converting datetime
#UFO_Dataframe['datetime'] = pd.to_datetime(UFO_Dataframe['datetime'])

# Separating the Date and Time can help in effective data anlysis, thus we separating date and time
UFO_Dataframe['datetime'] = UFO_Dataframe['datetime'].apply(lambda x: x.split())
UFO_Dataframe['date'] = UFO_Dataframe['datetime'].apply(lambda x: x[0])
UFO_Dataframe['time'] = UFO_Dataframe['datetime'].apply(lambda x: x[1])
del(UFO_Dataframe['datetime'])

UFO_Dataframe['date'] = pd.to_datetime(UFO_Dataframe['date'])
UFO_Dataframe['next_day'] = (UFO_Dataframe['time'] == '24:00') # rows where time = 24:00
UFO_Dataframe.loc[UFO_Dataframe['next_day'] == True, 'date'] = UFO_Dataframe['date'] + pd.Timedelta(days=1) # add a day
UFO_Dataframe.loc[UFO_Dataframe['next_day'] == True, 'time'] = '00:00' # set time to 00:00
del(UFO_Dataframe['next_day'])
UFO_Dataframe['time'] = pd.to_datetime(UFO_Dataframe['time']).dt.time

UFO_Dataframe['date posted'] = pd.to_datetime(UFO_Dataframe['date posted'])

del(UFO_Dataframe["duration (hours/min)"]) # duration(seconds) contains the same info in more readable form 

UFO_Dataframe.columns = ["City", "State", "Country", "Shape", "Duration (seconds)", "Notes/Comments", "Date Posted", "Latitude", "Longitude", "Date Observed", "Time Observed"]

UFO_Dataframe = UFO_Dataframe[["Date Observed", "Time Observed", "City", "State", "Country", "Latitude", "Longitude", "Date Posted", "Shape", "Duration (seconds)", "Notes/Comments"]]

UFO_Dataframe.head()

In [10]:
# one of the latitudes had invalid data so finding its index and dropping that row 
UFO_Dataframe.loc[UFO_Dataframe['latitude'] == '33q.200088']
UFO_Dataframe = UFO_Dataframe.drop(43782)

In [5]:
#Distribution of sightings across the world
#gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df['longitude '], df['latitude']))
#world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

#ax = world.plot(color='white', edgecolor='black', figsize=(35, 35))

# We can now plot our ``GeoDataFrame``.
#gdf.plot(ax=ax, color='red', markersize=2,legend=True)

#plt.show()

In [6]:
df.head(500)

,city,state,country,shape,duration (seconds),comments,date posted,latitude,longitude,date,time
0,san marcos,tx,us,cylinder,2700,This event took place in early fall around 194...,2004-04-27,29.8830556,-97.941111,1949-10-10,20:30:00
1,lackland afb,tx,unknown,light,7200,1949 Lackland AFB&#44 TX. Lights racing acros...,2005-12-16,29.38421,-98.581082,1949-10-10,21:00:00
2,chester (uk/england),unknown,gb,circle,20,Green/Orange circular disc over Chester&#44 En...,2008-01-21,53.2,-2.916667,1955-10-10,17:00:00
3,edna,tx,us,circle,20,My older brother and twin sister were leaving ...,2004-01-17,28.9783333,-96.645833,1956-10-10,21:00:00
4,kaneohe,hi,us,light,900,AS a Marine 1st Lt. flying an FJ4B fighter/att...,2004-01-22,21.4180556,-157.803611,1960-10-10,20:00:00
...,...,...,...,...,...,...,...,...,...,...,...
495,dushore,pa,us,disk,300,It could move very slowly or very quickly but ...,2005-04-16,41.5241667,-76.401111,1965-10-01,21:00:00
496,edmonton (30 mi. east of&#44 hwy 16 east) (can...,ab,ca,diamond,300,Unusual ights in distance. Approached until ob...,2001-08-05,53.55,-113.500000,1966-10-01,19:00:00
497,buffalo,ny,us,light,240,One of the bright stars moved north&#44 curved...,2002-01-11,42.8863889,-78.878611,1966-10-01,20:00:00
498,anacortes,wa,us,light,1200,20 miles driving toward Anacortes&#44a friend ...,2000-04-15,48.5127778,-122.611389,1966-10-01,22:00:00
